In [ ]:
#connecting colab with google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Path variables where training and testing datasets are

train_dir = "/content/drive/MyDrive/traffic-image/train"
val_dir   = "/content/drive/MyDrive/traffic-image/val"


In [ ]:
#listing training directories

import os
os.listdir(train_dir)

['priority-road',
 'no-entry',
 'construction-work',
 'speed-limit-50',
 'stop',
 'wild-animal-crossing']

In [ ]:
#creating directories

base_project = "/content/drive/MyDrive/traffic_project"
os.makedirs(base_project + "/models", exist_ok=True)
os.makedirs(base_project + "/notebooks", exist_ok=True)
os.makedirs(base_project + "/results", exist_ok=True)

In [ ]:
#importing libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 6817 images belonging to 6 classes.
Found 1705 images belonging to 6 classes.


In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 455s 2s/step - accuracy: 0.7705 - loss: 0.6615 - val_accuracy: 0.9578 - val_loss: 0.1174
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.9410 - loss: 0.1614 - val_accuracy: 0.9789 - val_loss: 0.0757
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 435s 2s/step - accuracy: 0.9598 - loss: 0.1154 - val_accuracy: 0.9812 - val_loss: 0.0544
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.9628 - loss: 0.1012 - val_accuracy: 0.9742 - val_loss: 0.0592
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9643 - loss: 0.0956 - val_accuracy: 0.9789 - val_loss: 0.0539
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 447s 2s/step - accuracy: 0.9713 - loss: 0.0808 - val_accuracy: 0.9806 - val_loss: 0.0493
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9699 - loss: 0.0853 - val_accuracy: 0.9877 - val_loss: 0.0404
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9718 - loss: 0.0760 - val_accu

In [ ]:
model.save("/content/drive/MyDrive/traffic_project/models/traffic_model.h5")

In [ ]:
model.save("/content/drive/MyDrive/traffic_project/models/traffic_model.keras")